In [14]:
#!pip install numpy
#!pip install torch
#!pip install transformers
#!pip install --upgrade pandas
#!pip install nltk
#!pip install git+https://github.com/Maluuba/nlg-eval.git@master
#!nlg-eval --setup

In [11]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM

from transformers_utils import load_json_QuAD_v1

import pandas as pd
import torch
import tqdm 

import regex as re
import nltk
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
import difflib

pd.set_option('display.max_colwidth', None)

In [30]:
# LOADING DATA
df_squad = load_json_QuAD_v1(
    "../SQuAD-data/dev-v1.1.json"
)
df_squad.head()

,id_question,question,answer_span,answer_span_start,id_context,context,doc_title
0,56be4db0acb8001400a502ec,Which NFL team represented the AFC at Super Bowl 50?,Denver Broncos,177,0,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.",Super_Bowl_50
1,56be4db0acb8001400a502ed,Which NFL team represented the NFC at Super Bowl 50?,Carolina Panthers,249,0,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.",Super_Bowl_50
2,56be4db0acb8001400a502ee,Where did Super Bowl 50 take place?,"Santa Clara, California",403,0,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.",Super_Bowl_50
3,56be4db0acb8001400a502ef,Which NFL team won Super Bowl 50?,Denver Broncos,177,0,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.",Super_Bowl_50
4,56be4db0acb8001400a502f0,What color was used to emphasize the 50th anniversary of the Super Bowl?,gold,488,0,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) c

## Hugging Face model

In [50]:
# LOADING MODEL
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

AttributeError: 'MarianTokenizer' object has no attribute 'vocab'

In [100]:
def trp(l, n, pad):
    """ Truncate or pad a list """
    r = l[:n]
    if len(r) < n:
        r.extend([pad] * (n - len(r)))
    return r

def translate(
    model, 
    tokenizer, 
    df, 
    batch_size, 
    device, 
    max_len_question, 
    max_len_answer, 
    max_len_context
):
    questions, contexts, answers = [], [], []
    batch_q, batch_a, batch_c = [], [], []
    new_context_history = []
    pad_token = tokenizer.pad_token_id
    curr_context = ''
    for iterrow in tqdm.tqdm(df.iterrows()):
        row = iterrow[1]
        
        if len(batch_q)==0:
            curr_context, new_context = row["context"], True
        else:
            new_context = (curr_context[:50] != row["context"][:50])
        if new_context:
            curr_context = row["context"]
            new_context_history.append(len(batch_q))
            input_context = trp(tokenizer.encode(row["context"]), max_len_question, pad_token)
            batch_c.append(input_context)
            
        input_question = trp(tokenizer.encode(row["question"]), max_len_question, pad_token)   
        input_answer = trp(tokenizer.encode(row["answer_span"]), max_len_question, pad_token)
        batch_q.append(input_question)
        batch_a.append(input_answer)
        
        if len(batch_q) == batch_size:
            batch_question = torch.tensor(batch_q).to(device)
            batch_answer = torch.tensor(batch_a).to(device)
            batch_context = torch.tensor(batch_c).to(device)
            batch_q, batch_a, batch_c = [], [], []
            
            generated_questions = model.generate(batch_question)
            generated_answers = model.generate(batch_answer)
            generated_contexts = model.generate(batch_context, min_length= max_len_context, max_length=max_len_context)
            
            index, completed = 0, False
            for i in range(batch_size):  
                if not completed:
                    if i >= new_context_history[index] :
                        index_bis = index
                        index += 1
                        if index == len(new_context_history): completed=True
                questions.append(tokenizer.decode(generated_questions[i], skip_special_tokens=True))
                contexts.append(tokenizer.decode(generated_contexts[index_bis], skip_special_tokens=True))
                answers.append(tokenizer.decode(generated_answers[i], skip_special_tokens=True))
                
            new_context_history = []
        if len(questions) > 100:
            break
    translated = pd.DataFrame({"question": questions, "answer": answers, "context":contexts})
    return translated

df_new = translate(model, tokenizer, df_squad, 32, device, 30, 15, 512)

127it [04:48,  2.27s/it]


In [86]:
df_squad.head(50)

,id_question,question,answer_span,answer_span_start,id_context,context,doc_title
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?,Saint Bernadette Soubirous,515,0,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",University_of_Notre_Dame
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,a copper statue of Christ,188,0,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",University_of_Notre_Dame
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame is beside to which structure?,the Main Building,279,0,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",University_of_Notre_Dame
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,381,0,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",University_of_Notre_Dame
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre Dame?,a golden statue of the Virgin Mary,92,0,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection

In [99]:
df_new.head(20)

,question,answer,context
0,À qui la Vierge Marie serait apparue en 1858 à Lourdes (France) ?,Sainte Bernadette Soubirous,"D'un point de vue architectural, l ""école a un caractère catholique. En haut du dôme d'or du bâtiment principal se trouve une statue dorée de la Vierge Marie. Immédiatement devant le bâtiment principal et en face de celui-ci se trouve une statue en cuivre du Christ avec des armes vantées par la légende"" Venite Ad Me Omnes "". À côté du bâtiment principal se trouve la basilique du Sacré-Cœur. Immédiatement derrière la basilique se trouve la Grotte, un lieu marial de prière et de réflexion. Il s'agit d'une réplique de la grotte de Lourdes, en France, où la Vierge Marie est censée être apparue à Sainte Bernadette Soubirous en 1858. Au bout de la promenade principale (et en ligne directe qui relie trois statues et le dôme d'or), se trouve une simple et moderne statue de Marie."
1,"Qu'est-ce qui se trouve devant l ""édifice Notre-Dame ?",une statue de cuivre du Christ,"D'un point de vue architectural, l ""école a un caractère catholique. En haut du dôme d'or du bâtiment principal se trouve une statue dorée de la Vierge Marie. Immédiatement devant le bâtiment principal et en face de celui-ci se trouve une statue en cuivre du Christ avec des armes vantées par la légende"" Venite Ad Me Omnes "". À côté du bâtiment principal se trouve la basilique du Sacré-Cœur. Immédiatement derrière la basilique se trouve la Grotte, un lieu marial de prière et de réflexion. Il s'agit d'une réplique de la grotte de Lourdes, en France, où la Vierge Marie est censée être apparue à Sainte Bernadette Soubirous en 1858. Au bout de la promenade principale (et en ligne directe qui relie trois statues et le dôme d'or), se trouve une simple et moderne statue de Marie."
2,La basilique du cœur sacré à Notre-Dame est à côté de quelle structure ?,le bâtiment principal,"D'un point de vue architectural, l ""école a un caractère catholique. En haut du dôme d'or du bâtiment principal se trouve une statue dorée de la Vierge Marie. Immédiatement devant le bâtiment principal et en face de celui-ci se trouve une statue en cuivre du Christ avec des armes vantées par la légende"" Venite Ad Me Omnes "". À côté du bâtiment principal se trouve la basilique du Sacré-Cœur. Immédiatement derrière la basilique se trouve la Grotte, un lieu marial de prière et de réflexion. Il s'agit d'une réplique de la grotte de Lourdes, en France, où la Vierge Marie est censée être apparue à Sainte Bernadette Soubirous en 1858. Au bout de la promenade principale (et en ligne directe qui relie trois statues et le dôme d'or), se trouve une simple et moderne statue de Marie."
3,Qu'est-ce que la Grotte de Notre-Dame ?,a Lieu marial de prière et de réflexion,"D'un point de vue architectural, l ""école a un caractère catholique. En haut du dôme d'or du bâtiment principal se trouve une statue dorée de la Vierge Marie. Immédiatement devant le bâtiment principal et en face de celui-ci se trouve une statue en cuivre du Christ avec des armes vantées par la légende"" Venite Ad Me Omnes "". À côté du bâtiment principal se trouve la basilique du Sacré-Cœur. Immédiatement derrière la basilique se trouve la Grotte, un lieu marial de prière et de réflexion. Il s'agit d'une réplique de la grotte de Lourdes, en France, où la Vierge Marie est censée être apparue à Sainte Bernadette Soubirous en 1858. Au bout de la promenade principale (et en ligne directe qui relie trois statues et le dôme d'or), se trouve une simple et moderne statue de Marie."
4,"Qu'est-ce qui se trouve au-dessus de l ""édifice principal de Notre-Dame ?",une statue en or de la Vierge Marie,"D'un point de vue architectural, l ""école a un caractère catholique. En haut du dôme d'or du bâtiment principal se trouve une statue dorée de la Vierge Marie. Immédiatement devant le bâtiment principal et en face de celui-ci se trouve une statue en cuivre du Christ avec des armes vantées par la légende"" Venite Ad Me Omnes "". À côté du b

## Fairseq model

In [3]:
!pip install fastBPE regex requests sacremoses subword_nmt

  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp36-cp36m-linux_x86_64.whl size=458459 sha256=4a4328f5f7a2f2e16018c0b3f07179be0dc9acb7aec62985c7ecb9567bb3a857
  Stored in directory: /home/azureuser/.cache/pip/wheels/df/60/ff/1764bce64cccd9d2c06ba19e5f6f4108ad29e2d48e1068c684
Successfully built fastBPE


In [4]:
en2fr = torch.hub.load(
    'pytorch/fairseq', 
    'transformer.wmt14.en-fr',
    tokenizer='moses', 
    bpe='subword_nmt', 
    #max_target_positions = 2048, 
    #max_input_positions = 2048,
    #decoder_input_dim = 2048,
    #decoder_output_dim = 2048,
    #max_source_positions=2048,
    )

# Use the GPU (optional):
en2fr.cuda()
#en2fr.args.max_target_positions = 2048
#en2fr.args.max_input_positions = 2048

print(en2fr.args)

Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /home/azureuser/.cache/torch/hub/master.zip


running build_ext
cythoning fairseq/data/data_utils_fast.pyx to fairseq/data/data_utils_fast.cpp
cythoning fairseq/data/token_block_utils_fast.pyx to fairseq/data/token_block_utils_fast.cpp
building 'fairseq.libbleu' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/fairseq
creating build/temp.linux-x86_64-3.6/fairseq/clib
creating build/temp.linux-x86_64-3.6/fairseq/clib/libbleu
gcc -pthread -B /anaconda/envs/azureml_py36/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/anaconda/envs/azureml_py36/include/python3.6m -c fairseq/clib/libbleu/libbleu.cpp -o build/temp.linux-x86_64-3.6/fairseq/clib/libbleu/libbleu.o -std=c++11 -O3 -DTORCH_API_INCLUDE_EXTENSION_H -DTORCH_EXTENSION_NAME=libbleu -D_GLIBCXX_USE_CXX11_ABI=1
gcc -pthread -B /anaconda/envs/azureml_py36/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/anaconda/envs/azure

100%|██████████| 2316140317/2316140317 [03:42<00:00, 10409149.51B/s]


Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='transformer_vaswani_wmt_en_de_big', attention_dropout=0.0, bpe='subword_nmt', bpe_codes='/home/azureuser/.cache/torch/pytorch_fairseq/53f403ba27ab138b06c1a8d78f5bb4f1722567ac3d3b3e41f821ec2cae2974da.7ef8ab763efda16d3c82dd8b5a574bdfe524e078bac7b444ea1a9c5d355b55ae/bpecodes', bpe_separator='@@', clip_norm=0.0, criterion='label_smoothed_cross_entropy', cross_self_attention=False, data='/home/azureuser/.cache/torch/pytorch_fairseq/53f403ba27ab138b06c1a8d78f5bb4f1722567ac3d3b3e41f821ec2cae2974da.7ef8ab763efda16d3c82dd8b5a574bdfe524e078bac7b444ea1a9c5d355b55ae', decoder_attention_heads=16, decoder_embed_dim=1024, decoder_embed_path=None, decoder_ffn_embed_dim=4096, decoder_input_dim=1024, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=Fals

In [ ]:
# Translate with beam search:
#fr = en2fr.translate(["""Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.""",
#                      "Hello, what's up?"], beam=5)
#print(fr)

## Manually tokenize:
#en_toks = en2fr.tokenize("""As at most other universities, Notre Dame's students run a number of news media outlets. The nine student-run outlets include three newspapers, both a radio and television station, and several magazines and journals. Begun as a one-page journal in September 1876, the Scholastic magazine is issued twice monthly and claims to be the oldest continuous collegiate publication in the United States. The other magazine, The Juggler, is released twice a year and focuses on student literature and artwork. The Dome yearbook is published annually. The newspapers have varying publication interests, with The Observer published daily and mainly reporting university and other news, and staffed by students from both Notre Dame and Saint Mary's College. Unlike Scholastic and The Dome, The Observer is an independent publication and does not have a faculty advisor or any editorial oversight from the University. In 1987, when some students believed that The Observer began to show a conservative bias, a liberal newspaper, Common Sense was published. Likewise, in 2003, when other students believed that the paper showed a liberal bias, the conservative paper Irish Rover went into production. Neither paper is published as often as The Observer; however, all three are distributed to all students. Finally, in Spring 2008 an undergraduate journal for political science research, Beyond Politics, made its debut.""")

# Manually apply BPE:
#en_bpe = en2fr.apply_bpe(en_toks)

# Manually binarize:
#en_bin1 = en2fr.binarize(en_bpe)
#en_bin2 = en2fr.binarize(en_bpe)
#print(en_bin.shape)
# Generate five translations with top-k sampling:
#fr_bin = en2fr.generate([en_bin1, en_bin2], beam=1, sampling=True, sampling_topk=1)
#print(fr_bin[0]['tokens'].shape)
#print(fr_bin) #batch, beams, dict

# Convert one of the samples to a string and detokenize
#fr_sample = fr_bin[0]['tokens']
#print(fr_sample.shape)
#fr_bpe = en2fr.string(fr_sample)
#fr_toks = en2fr.remove_bpe(fr_bpe)
#fr = en2fr.detokenize(fr_toks)
#print(fr)
#assert fr == en2fr.decode(fr_sample)
#print(en2fr.max_positions)

In [5]:
def translate_torch_v1(
    model, 
    df, 
    batch_size, 
):
    questions, contexts, answers = [], [], []
    batch_q, batch_a, batch_c = [], [], []
    new_context_history = []
    curr_context = ''
    for iterrow in tqdm.tqdm(df.iterrows()):
        row = iterrow[1]
        
        if len(batch_q)==0:
            curr_context, new_context = row["context"], True
        else:
            new_context = (curr_context[:50] != row["context"][:50])
        if new_context:
            curr_context = row["context"]
            new_context_history.append(len(batch_q))
            input_context = row["context"]
            batch_c.append(input_context)
            
        input_question = row["question"] 
        input_answer = row["answer_span"]
        batch_q.append(input_question)
        batch_a.append(input_answer)
        
        if len(batch_q) == batch_size:
            generated_questions = model.translate(batch_q)
            generated_answers = model.translate(batch_a)
            generated_contexts = model.translate(batch_c)          
            batch_q, batch_a, batch_c = [], [], []
            
            index, completed = 0, False
            for i in range(batch_size):  
                if not completed:
                    if i >= new_context_history[index] :
                        index_bis = index
                        index += 1
                        if index == len(new_context_history): completed=True
                questions.append(generated_questions[i])
                contexts.append(generated_contexts[index_bis])
                answers.append(generated_answers[i])
            new_context_history = []
        if len(questions) > 100:
            break
    translated = pd.DataFrame({"question": questions, "answer": answers, "context":contexts})
    return translated

def translate_torch_v2(
    model, 
    df, 
    batch_size, 
):
    questions, contexts, answers = [], [], []
    batch_q, batch_a, batch_c, batch_c_bis = [], [], [], []
    new_context_history, division_history = [], []
    curr_context = ''
    for iterrow in tqdm.tqdm(df.iterrows()):
        row = iterrow[1]
        # we avoid translating 2 times the same context
        if len(batch_q)==0:
            curr_context, new_context = row["context"], True
        else:
            new_context = (curr_context[:50] != row["context"][:50])
        if new_context:
            curr_context = row["context"]
            new_context_history.append(len(batch_q))
            input_context = row["context"]
            
            bin_context = model.binarize(model.apply_bpe(model.tokenize(input_context)))
            # if the sentence is too long, we split it into two parts.
            if bin_context.shape[0] > 200:
                sentences = input_context.split(". ")
                separator = len(sentences)//2 + 1
                part1 = model.binarize(model.apply_bpe(model.tokenize(". ".join(sentences[:separator]))))
                part2 = model.binarize(model.apply_bpe(model.tokenize(". ".join(sentences[separator:]))))
                batch_c.append(part1)
                batch_c_bis.append(part2)
                # we note that this context was split
                division_history.append(len(batch_q))
            else: 
                batch_c.append(bin_context)
            
        input_question = row["question"] 
        input_answer = row["answer_span"]
        batch_q.append(model.binarize(model.apply_bpe(model.tokenize(input_question))))
        batch_a.append(model.binarize(model.apply_bpe(model.tokenize(input_answer))))
        
        if len(batch_q) == batch_size:
            generated_questions = model.generate(batch_q, beam=1) #, sampling=False, sampling_topk=1)
            generated_answers = model.generate(batch_a, beam=1) #, sampling=False, sampling_topk=1)
            generated_contexts = model.generate(batch_c, beam=1) #, sampling=False, sampling_topk=1)   
            generated_contexts_bis = model.generate(batch_c_bis, beam=1) #, sampling=False, sampling_topk=1) 
            
            batch_q, batch_a, batch_c, batch_c_bis = [], [], [], []
            
            index, index_split, completed = 0, -1, False
            for i in range(batch_size): 
                splited = False
                if not completed:
                    if i >= new_context_history[index]:
                        # if the context was splited:
                        if new_context_history[index] in division_history:
                            splited = True
                            index_split += 1
                        index_context = index
                        index += 1
                        if index == len(new_context_history): completed=True
                questions.append(model.detokenize(model.remove_bpe(model.string(generated_questions[i][0]['tokens'])))) 
                context = model.detokenize(model.remove_bpe(model.string(generated_contexts[index_context][0]['tokens']))) +\
                    (" " + model.detokenize(model.remove_bpe(model.string(generated_contexts_bis[index_split][0]['tokens']))) if splited else "")
                contexts.append(context)
                answers.append(model.detokenize(model.remove_bpe(model.string(generated_answers[i][0]['tokens']))))
            new_context_history = []
            division_history = []
    translated = pd.DataFrame({"question": questions, "answer": answers, "context":contexts})
    return translated

In [ ]:
df_new = translate_torch_v2(en2fr, df_squad, 32)
#df_new.to_csv("../FQuAD-data/translated-train.csv", sep=",")

64it [00:17,  3.76it/s]

In [ ]:
df_squad.head(20)

In [ ]:
df_new.head(20)

## Extract answer span

In [33]:
df_new = pd.read_csv("../FQuAD-data/translated-valid.csv")

In [34]:
df_new.head(20)

,Unnamed: 0,Unnamed: 0.1,question,answer,context
0,0,0,Quelle équipe de la NFL représentait le CFA au Super Bowl 50 ?,Denver Broncos 2.,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50."
1,1,1,Quelle équipe de la NFL représentait le NFC au Super Bowl 50 ?,Carolina Panthers,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50."
2,2,2,Où s'est déroulé le Super Bowl 50 ?,Santa Clara (Californie),"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50."
3,3,3,Quelle équipe de la NFL a remporté le Super Bowl 50 ?,Denver Broncos 2.,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50."
4,4,4,Quelle couleur a été utilisée pour souligner le 50e anniversaire du Super Bowl ?,or,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte 

In [35]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

#list(find_all('spam spam spam spam', 'spam'))

def untokenize(words):
    """
    Untokenizing a text undoes the tokenizing operation, restoring
    punctuation and spaces to the places that people expect them to be.
    Ideally, `untokenize(tokenize(text))` should be identical to `text`,
    except for line breaks.
    """
    text = ' '.join(words)
    step1 = text.replace("`` ", '"').replace(" ''", '"').replace('. . .',  '...')
    step2 = step1.replace("( ", "(").replace(" )", ")")
    step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
    step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
    step5 = step4.replace(" '", "'").replace("' ", "'").replace(
         "can not", "cannot")
    step6 = step5.replace(" ` ", " '")
    return step6.strip()

def strip_string(string_):
    # strip start
    if len(string_) == 0: return string_
    alphanum, possible = False, True
    index = -1
    while alphanum == False and possible:
        index += 1
        alphanum = string_[index].isalnum()
        possible = index < len(string_) - 1
    string_ = string_[index:]
    alphanum = False
    index = 0
    while alphanum == False and possible:
        index += 1
        alphanum = string_[-index].isalnum()
        possible = index < len(string_)
    string_ = string_[:-index+1] if index > 1 else string_
    return string_

In [36]:
def extract_answer_span(df_new, df_old):
    """
    This function will search for the position of the text span inside the context.
    If the words are exactly inside the context, it will be easy, if not, we will 
    replace the text span by a span of the same length close to the translated answer,
    among all the texts.
    INPUTS:
    - df_new: pandas dataframe, translated squad
    - df_old: pandas dataframe, original squad.
    OUTPUTS:
    - starts: list, answer_span_start
    - to_drop: list, indexes of rows to delete.
    """
    count = 0
    starts = []
    to_drop = []
    for i, iterrow in tqdm.tqdm(enumerate(df_new.iterrows())):
        dropping = False
        row_old = df_old.iloc[i]
        answer_old = row_old["answer_span"]
        context_old = row_old["context"]
        
        row = iterrow[1]
        try: 
            answer = strip_string(row["answer"])
        except:
            to_drop.append(i)
            dropping = True
            continue
            
        context = row["context"]
        # let's try to find it in french
        positions = list(find_all(context, answer))
        if positions != []:
            if len(positions) == 1:
                start = positions[0]
            if len(positions) > 1:
                positions_old = list(find_all(context_old, answer_old))
                start_old = row_old["answer_span_start"]
                index = positions_old.index(start_old)
                if len(positions_old) == len(positions):
                    start = positions[index]
                else:
                    #start = elem in positions closest to start_old
                    start = positions[np.argmin([abs(position-start_old) for position in positions])]
        else:
            # let's try to find it in english
            positions_bis = list(find_all(context, answer_old)) 
            if len(positions_bis) == 1:
                start = positions_bis[0]
                
            if len(positions_bis) > 1:
                positions_old = list(find_all(context_old, answer_old))
                start_old = row_old["answer_span_start"]
                index = positions_old.index(start_old)
                if len(positions_old) == len(positions_bis):
                    start = positions_bis[index]
                else:
                    #start = elem in positions closest to start_old
                    start = positions_bis[np.argmin([abs(position-start_old) for position in positions_bis])]
            
            if len(positions_bis)==0:
                #compare the scores in english and in french.
                
                answer_tokenized = nltk.word_tokenize(answer)
                answer_old_tokenized = nltk.word_tokenize(answer_old)
                context_tokenized = nltk.word_tokenize(context)
                len_answer = len(answer_tokenized)
                len_answer_old = len(answer_old_tokenized)
                len_context = len(context_tokenized)
                # with difflib
                all_text_spans = [untokenize(context_tokenized[k:k + len_answer]) for k in range(len_context - len_answer + 1)]
                french_match = difflib.get_close_matches(answer, all_text_spans)
                french_similarity_found = len(french_match) > 0
                
                all_text_spans_en = [untokenize(context_tokenized[k:k + len_answer_old]) for k in range(len_context - len_answer_old + 1)]
                english_match = difflib.get_close_matches(answer_old, all_text_spans_en)
                english_similarity_found = len(english_match) > 0
                
                score_french, score_english = 0, 0
                if french_similarity_found or english_similarity_found:
                    if french_similarity_found:
                        score_french = difflib.SequenceMatcher(None, answer, french_match[0]).ratio()
                    if english_similarity_found:
                        score_english = difflib.SequenceMatcher(None, answer_old, english_match[0]).ratio()
                    text_span = french_match[0] if score_french >= score_english else english_match[0]
                    text_span = strip_string(text_span)
                    try:
                        start = list(find_all(untokenize(context_tokenized), text_span))[0]
                    except:
                        to_drop.append(i)
                        dropping = True

                # with BLEU
                #all_text_spans = [context_tokenized[k:k + len_answer] for k in range(len_context - len_answer + 1)]
                #scores = []
                #similarity_found = False
                #for hypothesis in all_text_spans:
                    #score = sentence_bleu(answer_tokenized, hypothesis)
                    #if score > 0:
                    #    similarity_found = True
                    #scores.append(score)
                #if similarity_found:
                #    index = np.argmax(scores)
                #    text_span = untokenize(context_tokenized[index:index + len_answer])
                #    start = list(find_all(context, text_span))[0]

                else:
                    # soit on laisse inchangé... mauvaise idée... soit on prend le text span à la position
                    # de l'anglais, soit on supprime celui là. 
                    #text_span = untokenize(nltk.word_tokenize(context[start_old:])[:len(answer_tokenized)])
                    #start = list(find_all(context, text_span))[0]
                    to_drop.append(i)
                    dropping = True
        if not dropping:
            starts.append(start)
    return starts, to_drop
span_starts, to_drop = extract_answer_span(df_new, df_squad)
df_bis = df_new.drop(to_drop)
df_bis = df_bis.reset_index()

10560it [01:45, 99.74it/s] 


In [37]:
df_bis["answer_span_start"] = span_starts

In [38]:
df_bis = df_bis.rename(columns={"answer": "answer_span"})

In [39]:
try:
    df_bis = df_bis.drop(columns=["index"])
except:
    pass
try:
    df_bis = df_bis.drop(columns=["Unnamed: 0"])
except:
    pass
try:
    df_bis = df_bis.drop(columns=["Unnamed: 0.1"])
except:
    pass

In [40]:
df_bis.head(20)

,question,answer_span,context,answer_span_start
0,Quelle équipe de la NFL représentait le CFA au Super Bowl 50 ?,Denver Broncos 2.,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",52
1,Quelle équipe de la NFL représentait le NFC au Super Bowl 50 ?,Carolina Panthers,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",130
2,Où s'est déroulé le Super Bowl 50 ?,Santa Clara (Californie),"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",315
3,Quelle équipe de la NFL a remporté le Super Bowl 50 ?,Denver Broncos 2.,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",52
4,Quelle couleur a été utilisée pour souligner le 50e anniversaire du Super Bowl ?,or,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le 

In [41]:
def post_proc(df):
    df_copy = df.copy()
    answers_bis = []
    end_chars = [".", ",", ":", ";", " ", "-", "'", '"', ")", "`"]
    for i, row in df.iterrows():
        answer = strip_string(row["answer_span"])
        context = row["context"]
        answer_span_start = row["answer_span_start"]
        len_max = len(context)
        stop = False
        j = 0
        while not stop and len(answer) + answer_span_start + j < len_max:
            stop = context[len(answer) + answer_span_start + j] in end_chars
            j += 1
            
        new_answer_span = context[answer_span_start:len(answer) + answer_span_start + j-1]
            
        answers_bis.append(new_answer_span)
    df_copy["answer_span_bis"] = answers_bis
    return df_copy
df_bis = post_proc(df_bis)

In [42]:
df_bis.head(40)

,question,answer_span,context,answer_span_start,answer_span_bis
0,Quelle équipe de la NFL représentait le CFA au Super Bowl 50 ?,Denver Broncos 2.,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",52,Denver Broncos a
1,Quelle équipe de la NFL représentait le NFC au Super Bowl 50 ?,Carolina Panthers,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",130,Carolina Panthers
2,Où s'est déroulé le Super Bowl 50 ?,Santa Clara (Californie),"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",315,"Clara, en Californie. Comme"
3,Quelle équipe de la NFL a remporté le Super Bowl 50 ?,Denver Broncos 2.,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres romains (sous lesquels le jeu aurait été connu sous le nom de ""Super Bowl L""), de sorte que le logo puisse mettre en évidence les chiffres arabes 50.",52,Denver Broncos a
4,Quelle couleur a été utilisée pour souligner le 50e anniversaire du Super Bowl ?,or,"Le champion de l'American Football Conference (AFC) Denver Broncos a battu la championne de la National Football Conference (NFC) Carolina Panthers 24 * * 10 pour remporter son troisième titre de Super Bowl. La partie a eu lieu le 7 février 2016 au Levi's Stadium dans la région de la baie de San Francisco à Santa Clara, en Californie. Comme il s'agissait du 50e Super Bowl, la ligue a souligné le ""50e anniversaire d'or"" par diverses initiatives ayant un thème d'or, et a suspendu temporairement la tradition de nommer chaque match du Super Bowl par des chiffres

In [43]:
df_bis.to_csv("../FQuAD-data/translated-valid_v1.1.csv", sep=",")

In [29]:
print(df_bis)

                                                                             question  \
0                  À qui la Vierge Marie serait apparue en 1858 à Lourdes en France ?   
1                                         Que se trouve devant l'édifice Notre-Dame ?   
2           La basilique du cœur sacré de Notre-Dame est à côté de quelle structure ?   
3                                             Qu'est-ce que la Grotte de Notre Dame ?   
4            Qu'est-ce qui se trouve au-dessus de l'édifice principal de Notre-Dame ?   
...                                                                               ...   
77604  Quand l'Institute of Medicine a-t-il commencé à offrir des cours de médecine ?   
77605                                                          Que représente KUSMS ?   
77606       Avec le cricket, quel sport est très populaire chez les jeunes Népalais ?   
77607         Combien de personnes peuvent faire partie du stade Dasarath Rangasala ?   
77608         Dans qu